# Preparetion new COS for Molpher, with deleted scaffold what we don;t want from path

# Library import

In [2]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import AllChem
from rdkit.Chem.Scaffolds.MurckoScaffold import GetScaffoldForMol
from rdkit.Chem.Scaffolds.MurckoScaffold import MakeScaffoldGeneric
from rdkit.Chem.Scaffolds.MurckoScaffold import MurckoScaffoldSmiles
from rdkit.DataManip.Metric import GetTanimotoDistMat
from rdkit.DataManip.Metric import GetTanimotoSimMat
from rdkit.Chem import DataStructs
from rdkit.ML.Cluster import Butina
from rdkit.ML.Cluster.Butina import EuclideanDist
from rdkit.ML.KNN.DistFunctions import TanimotoDist


#descripts
from rdkit.Chem import Lipinski
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Descriptors

from rdkit.Chem.Lipinski import NumAromaticHeterocycles
from rdkit.Chem.Lipinski import NumAliphaticRings
from rdkit.Chem.Lipinski import NumAromaticHeterocycles
from rdkit.Chem.Lipinski import NumAromaticRings

from rdkit.Chem.SaltRemover import SaltRemover

from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw

from scipy.spatial import distance
from sklearn import manifold
from sklearn import decomposition
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
from sklearn_extra.cluster import KMedoids
from sklearn.metrics import matthews_corrcoef
import statistics

# plots
from matplotlib import pyplot
import seaborn

import itertools
from itertools import combinations

import numpy as np

from sklearn.metrics import jaccard_score

import os
import matplotlib.pyplot as plt

## Loading data

In [3]:
dont_wanted_scaffold = pd.read_csv("data/input_recall_sets/Glucocorticoid_receptor/new_sets/df_not_in_new_active_sets_new.csv")
dont_wanted_scaffold

,canonical_smiles,scaffolds_csk,pchembl_value
0,C=CCOC(C)C1=CC(C)(C)Nc2ccc(-c3cc(F)ccc3OC)cc21,C1CCC(C2CCC3CCCCC3C2)CC1,7.400000
1,Cc1c(NS(C)(=O)=O)cccc1N(Cc1ccccc1)Cc1ccc(Oc2cc...,C(CCCC1CCC(CC2CCC(CC(CC3CCCCC3)C3CCCCC3)CC2)CC...,7.770000
2,COc1cc(C(=O)c2cccc(C)c2)c(Oc2c(Br)cc(CC(=O)O)c...,C1CCC(CC2CCCCC2CC2CCCCC2)CC1,7.710000
3,Cc1noc(=O)c2ccc(NC(=O)C(O)(CC3(C4CCC4)CCCc4ccc...,C(CCC1(C2CCC2)CCCC2CCCCC21)CC1CCC2CCCCC2C1,8.165000
4,C[C@H](NC(=O)C(C)(F)F)[C@H](Oc1ccc2c(cnn2-c2cc...,C1CCC(CCC2CCC3C(CCC3C3CCCC(C4CC4)C3)C2)CC1,8.410000
5,Cc1c(NS(C)(=O)=O)cccc1N(Cc1ccc(Oc2cccc(OC[C@@H...,C1CCC(CCC2CCCC(CC3CCC(CC(CC4CCCCC4)C4CCCCC4)CC...,7.600000
6,Cc1c(NS(C)(=O)=O)cccc1N(Cc1ccccc1)Cc1ccc(Oc2cc...,C(CCCC1CCCC(CC2CCC(CC(CC3CCCCC3)C3CCCCC3)CC2)C...,7.240000
7,COc1cccc2c1-c1ccc(NS(C)(=O)=O)cc1C(c1cccc(Oc3c...,C1CCC(CC2CCCC(C3CC4CCCCC4C4CCCCC34)C2)CC1,7.320000
8,N#Cc1ccc(O[C@H](c2cnc(C3CC3)nc2)[C@@H](NC(=O)C...,C1CCC(CC(CC2CC2)C2CCC(C3CC3)CC2)CC1,7.765000
9,C[C@H](NC(=O)C(C)(F)F)[C@H](Oc1ccc2c(cnn2-c2cc...,C1CCC(CCC2CCC3C(CCC3C3CCCC(CC4CC4)C3)C2)CC1,8.360000


In [8]:
type = 'sim'
number = 0

In [9]:
df_output = pd.read_csv(f"data/output_sets/Glucocorticoid_receptor/cOS_Molpher_{type}_{number}_all_columns.csv", header = None)
df_output.columns = ['chembl_start', 'chembl_stop', 'start_smiles', 'stop_smiles', 'morph', 'tanimoto', 'length', 'time']
df_output['start_scaffold'] = [MurckoScaffoldSmiles(\
                                         Chem.MolToSmiles(MakeScaffoldGeneric\
                                       (Chem.MolFromSmiles(x)))) for x in df_output.start_smiles]
df_output['stop_scaffold'] = [MurckoScaffoldSmiles(\
                                         Chem.MolToSmiles(MakeScaffoldGeneric\
                                       (Chem.MolFromSmiles(x)))) for x in df_output.stop_smiles]
df_output

,chembl_start,chembl_stop,start_smiles,stop_smiles,morph,tanimoto,length,time,start_scaffold,stop_scaffold
0,CHEMBL3647440,CHEMBL3647439,Cc1cnc(NC(=O)c2ccc3c(c2)CC[C@@H]2C[C@@](O)(C(F...,Cc1ncccc1CNC(=O)c1ccc2c(c1)CC[C@@H]1C[C@@](O)(...,CC1=CN=C(NC(=O)C2=CC=C3C(=C2)CCC2CC(O)(C(F)(F)...,0.702303,10,31.867709,C1CCC(CCC2CCC3C(CCC4CCCCC43CC3CCCCC3)C2)CC1,C1CCC(CCCC2CCC3C(CCC4CCCCC43CC3CCCCC3)C2)CC1
1,CHEMBL3647440,CHEMBL3647439,Cc1cnc(NC(=O)c2ccc3c(c2)CC[C@@H]2C[C@@](O)(C(F...,Cc1ncccc1CNC(=O)c1ccc2c(c1)CC[C@@H]1C[C@@](O)(...,CC1=NC=CN=C1NC(=O)C1=CC=C2C(=C1)CCC1CC(O)(C(F)...,0.702303,10,31.867709,C1CCC(CCC2CCC3C(CCC4CCCCC43CC3CCCCC3)C2)CC1,C1CCC(CCCC2CCC3C(CCC4CCCCC43CC3CCCCC3)C2)CC1
2,CHEMBL3647440,CHEMBL3647439,Cc1cnc(NC(=O)c2ccc3c(c2)CC[C@@H]2C[C@@](O)(C(F...,Cc1ncccc1CNC(=O)c1ccc2c(c1)CC[C@@H]1C[C@@](O)(...,CC1=NC=CC=C1NC(=O)C1=CC=C2C(=C1)CCC1CC(O)(C(F)...,0.702303,10,31.867709,C1CCC(CCC2CCC3C(CCC4CCCCC43CC3CCCCC3)C2)CC1,C1CCC(CCCC2CCC3C(CCC4CCCCC43CC3CCCCC3)C2)CC1
3,CHEMBL3647440,CHEMBL3647439,Cc1cnc(NC(=O)c2ccc3c(c2)CC[C@@H]2C[C@@](O)(C(F...,Cc1ncccc1CNC(=O)c1ccc2c(c1)CC[C@@H]1C[C@@](O)(...,CC1=NC=CC=C1CNC(=O)C1=CC=C2C(=C1)CCC1CC(O)(C(F...,0.702303,10,31.867709,C1CCC(CCC2CCC3C(CCC4CCCCC43CC3CCCCC3)C2)CC1,C1CCC(CCCC2CCC3C(CCC4CCCCC43CC3CCCCC3)C2)CC1
4,CHEMBL3647440,CHEMBL3647439,Cc1cnc(NC(=O)c2ccc3c(c2)CC[C@@H]2C[C@@](O)(C(F...,Cc1ncccc1CNC(=O)c1ccc2c(c1)CC[C@@H]1C[C@@](O)(...,CC1=NC=CC=C1CNC(=O)C1=CC=C2C(=C1)CCC1CC(O)(C(F...,0.702303,10,31.867709,C1CCC(CCC2CCC3C(CCC4CCCCC43CC3CCCCC3)C2)CC1,C1CCC(CCCC2CCC3C(CCC4CCCCC43CC3CCCCC3)C2)CC1
...,...,...,...,...,...,...,...,...,...,...
1107570,CHEMBL235208,CHEMBL208840,COc1ccc(F)cc1C(C)(C)CC(O)(CNC(=O)c1cnn(-c2ccc(...,CCC1(CC(O)(C(=O)Nc2ccc3c(=O)onc(C)c3c2)C(F)(F)...,CCC1(CCC(O)(C(=O)NC2=CC=C3C(=O)ON=C(C)C3=C2)C(...,0.484868,38,202.314861,C(CCCC1CCC(C2CCCCC2)C1)CCC1CCCCC1,C(CCC1CCCCC2CCCCC12)CC1CCC2CCCCC2C1
1107571,CHEMBL235208,CHEMBL208840,COc1ccc(F)cc1C(C)(C)CC(O)(CNC(=O)c1cnn(-c2ccc(...,CCC1(CC(O)(C(=O)Nc2ccc3c(=O)onc(C)c3c2)C(F)(F)...,CCC1(CCC(O)(C(=O)NC2=CC=C3C(=O)ON=C(C)C3=C2)C(...,0.484868,38,202.314861,C(CCCC1CCC(C2CCCCC2)C1)CCC1CCCCC1,C(CCC1CCCCC2CCCCC12)CC1CCC2CCCCC2C1
1107572,CHEMBL235208,CHEMBL208840,COc1ccc(F)cc1C(C)(C)CC(O)(CNC(=O)c1cnn(-c2ccc(...,CCC1(CC(O)(C(=O)Nc2ccc3c(=O)onc(C)c3c2)C(F)(F)...,CCC1(CCC(O)(C(=O)NC2=CC=C3C(=O)ON=C(C)C3=C2)C(...,0.484868,38,202.314861,C(CCCC1CCC(C2CCCCC2)C1)CCC1CCCCC1,C(CCC1CCCCC2CCCCC12)CC1CCC2CCCCC2C1
1107573,CHEMBL235208,CHEMBL208840,COc1ccc(F)cc1C(C)(C)CC(O)(CNC(=O)c1cnn(-c2ccc(...,CCC1(CC(O)(C(=O)Nc2ccc3c(=O)onc(C)c3c2)C(F)(F)...,CCC1(CCC(O)(C(=O)NC2=CC=C3C(=O)ON=C(C)C3=C2)C(...,0.484868,38,202.314861,C(CCCC1CCC(C2CCCCC2)C1)CCC1CCCCC1,C(CCC1CCCCC2CCCCC12)CC1CCC2CCCCC2C1


In [10]:
drop_index = []
for scaffold in dont_wanted_scaffold.scaffolds_csk:
    arr_drop_id = df_output[df_output.start_scaffold == scaffold].index
    for x in arr_drop_id:
        if x not in drop_index:
            drop_index.append(x)
    arr_drop_id = df_output[df_output.stop_scaffold == scaffold].index
    for x in arr_drop_id:
        if x not in drop_index:
            drop_index.append(x)
print(len(drop_index)) #found index of rows wich consist of scaffold in Start or Stop molecule what we don't want
df_output_new = df_output.copy()
df_output_new = df_output_new.drop(drop_index)
df_output_new.reset_index(level=0, inplace=True, drop = True)
df_output_new

251611


,chembl_start,chembl_stop,start_smiles,stop_smiles,morph,tanimoto,length,time,start_scaffold,stop_scaffold
0,CHEMBL3647440,CHEMBL3647439,Cc1cnc(NC(=O)c2ccc3c(c2)CC[C@@H]2C[C@@](O)(C(F...,Cc1ncccc1CNC(=O)c1ccc2c(c1)CC[C@@H]1C[C@@](O)(...,CC1=CN=C(NC(=O)C2=CC=C3C(=C2)CCC2CC(O)(C(F)(F)...,0.702303,10,31.867709,C1CCC(CCC2CCC3C(CCC4CCCCC43CC3CCCCC3)C2)CC1,C1CCC(CCCC2CCC3C(CCC4CCCCC43CC3CCCCC3)C2)CC1
1,CHEMBL3647440,CHEMBL3647439,Cc1cnc(NC(=O)c2ccc3c(c2)CC[C@@H]2C[C@@](O)(C(F...,Cc1ncccc1CNC(=O)c1ccc2c(c1)CC[C@@H]1C[C@@](O)(...,CC1=NC=CN=C1NC(=O)C1=CC=C2C(=C1)CCC1CC(O)(C(F)...,0.702303,10,31.867709,C1CCC(CCC2CCC3C(CCC4CCCCC43CC3CCCCC3)C2)CC1,C1CCC(CCCC2CCC3C(CCC4CCCCC43CC3CCCCC3)C2)CC1
2,CHEMBL3647440,CHEMBL3647439,Cc1cnc(NC(=O)c2ccc3c(c2)CC[C@@H]2C[C@@](O)(C(F...,Cc1ncccc1CNC(=O)c1ccc2c(c1)CC[C@@H]1C[C@@](O)(...,CC1=NC=CC=C1NC(=O)C1=CC=C2C(=C1)CCC1CC(O)(C(F)...,0.702303,10,31.867709,C1CCC(CCC2CCC3C(CCC4CCCCC43CC3CCCCC3)C2)CC1,C1CCC(CCCC2CCC3C(CCC4CCCCC43CC3CCCCC3)C2)CC1
3,CHEMBL3647440,CHEMBL3647439,Cc1cnc(NC(=O)c2ccc3c(c2)CC[C@@H]2C[C@@](O)(C(F...,Cc1ncccc1CNC(=O)c1ccc2c(c1)CC[C@@H]1C[C@@](O)(...,CC1=NC=CC=C1CNC(=O)C1=CC=C2C(=C1)CCC1CC(O)(C(F...,0.702303,10,31.867709,C1CCC(CCC2CCC3C(CCC4CCCCC43CC3CCCCC3)C2)CC1,C1CCC(CCCC2CCC3C(CCC4CCCCC43CC3CCCCC3)C2)CC1
4,CHEMBL3647440,CHEMBL3647439,Cc1cnc(NC(=O)c2ccc3c(c2)CC[C@@H]2C[C@@](O)(C(F...,Cc1ncccc1CNC(=O)c1ccc2c(c1)CC[C@@H]1C[C@@](O)(...,CC1=NC=CC=C1CNC(=O)C1=CC=C2C(=C1)CCC1CC(O)(C(F...,0.702303,10,31.867709,C1CCC(CCC2CCC3C(CCC4CCCCC43CC3CCCCC3)C2)CC1,C1CCC(CCCC2CCC3C(CCC4CCCCC43CC3CCCCC3)C2)CC1
...,...,...,...,...,...,...,...,...,...,...
855959,CHEMBL235208,CHEMBL208840,COc1ccc(F)cc1C(C)(C)CC(O)(CNC(=O)c1cnn(-c2ccc(...,CCC1(CC(O)(C(=O)Nc2ccc3c(=O)onc(C)c3c2)C(F)(F)...,CCC1(CCC(O)(C(=O)NC2=CC=C3C(=O)ON=C(C)C3=C2)C(...,0.484868,38,202.314861,C(CCCC1CCC(C2CCCCC2)C1)CCC1CCCCC1,C(CCC1CCCCC2CCCCC12)CC1CCC2CCCCC2C1
855960,CHEMBL235208,CHEMBL208840,COc1ccc(F)cc1C(C)(C)CC(O)(CNC(=O)c1cnn(-c2ccc(...,CCC1(CC(O)(C(=O)Nc2ccc3c(=O)onc(C)c3c2)C(F)(F)...,CCC1(CCC(O)(C(=O)NC2=CC=C3C(=O)ON=C(C)C3=C2)C(...,0.484868,38,202.314861,C(CCCC1CCC(C2CCCCC2)C1)CCC1CCCCC1,C(CCC1CCCCC2CCCCC12)CC1CCC2CCCCC2C1
855961,CHEMBL235208,CHEMBL208840,COc1ccc(F)cc1C(C)(C)CC(O)(CNC(=O)c1cnn(-c2ccc(...,CCC1(CC(O)(C(=O)Nc2ccc3c(=O)onc(C)c3c2)C(F)(F)...,CCC1(CCC(O)(C(=O)NC2=CC=C3C(=O)ON=C(C)C3=C2)C(...,0.484868,38,202.314861,C(CCCC1CCC(C2CCCCC2)C1)CCC1CCCCC1,C(CCC1CCCCC2CCCCC12)CC1CCC2CCCCC2C1
855962,CHEMBL235208,CHEMBL208840,COc1ccc(F)cc1C(C)(C)CC(O)(CNC(=O)c1cnn(-c2ccc(...,CCC1(CC(O)(C(=O)Nc2ccc3c(=O)onc(C)c3c2)C(F)(F)...,CCC1(CCC(O)(C(=O)NC2=CC=C3C(=O)ON=C(C)C3=C2)C(...,0.484868,38,202.314861,C(CCCC1CCC(C2CCCCC2)C1)CCC1CCCCC1,C(CCC1CCCCC2CCCCC12)CC1CCC2CCCCC2C1


In [11]:
df_output_new.to_csv(f"data/output_sets/Glucocorticoid_receptor/new_sets/cOS_Molpher_{type}_{number}_all_columns.csv", index = False,index_label = False, header = False)